In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
from glob import glob
# Root directory of the project
# ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
# sys.path.append(ROOT_DIR)  # To find local version of the library
from src.config import Config
from src import harz
import src.model as modellib
import src.utils as utils

Using TensorFlow backend.


In [18]:
import imp
harz = imp.reload(harz)

In [17]:
class InferenceConfig(harz.HarzConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.5
config = InferenceConfig()

In [18]:
model = modellib.MaskRCNN(mode="inference", config=config,
                                  model_dir='./log')

In [15]:
weights_path = model.find_last()

In [5]:
weights_path

'./log/harz20190219T1448/mask_rcnn_harz_0300.h5'

In [19]:
model.load_weights(weights_path, by_name=True)

Re-starting from epoch 300


In [15]:
images

['/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05280.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05494.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05736.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05741.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm051012.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05888.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05177.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05613.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm051194.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05728.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05492.tif',
 '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/mar

In [20]:
r = harz.detect_and_color_splash(model, images[1])

Running on /notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05494.tif
Processing 1 images
image                    shape: (256, 256, 1)         min:    0.00000  max:    1.00000  float32
molded_images            shape: (1, 256, 256, 1)      min:    0.00000  max:    1.00000  float64
image_metas              shape: (1, 15)               min:    0.00000  max:  256.00000  int64
anchors                  shape: (1, 16368, 4)         min:   -1.41976  max:    2.16878  float32


In [21]:
r['scores']

array([0.6325773 , 0.63063186, 0.6283171 , 0.5682578 , 0.55886924,
       0.5548474 , 0.55292994, 0.5404167 , 0.5395594 , 0.5350639 ,
       0.5340026 , 0.5327204 , 0.53204376, 0.53164285, 0.5263841 ,
       0.524424  , 0.52259266, 0.52079636, 0.5203767 , 0.5151552 ,
       0.5131511 , 0.51123166, 0.50919676, 0.50802124, 0.50618476,
       0.5019827 ], dtype=float32)

In [22]:
out = np.zeros((len(r['scores']),256,256))
print(out.shape)
for i in range(len(r['scores'])):
    out[i][r['rois'][i][0]:r['rois'][i][2],r['rois'][i][1]:r['rois'][i][3]] = 1

(26, 256, 256)


In [24]:
from src import my_utils as ut

In [26]:
for i in range(5):
    ut.write_numpy_array_to_tif(out[i], 'cc'+str(i)+'.tif', original_tif=images[1])

In [19]:
r = harz.detect_and_color_splash(model, images[0])

Running on /notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm05280.tif
Processing 1 images
image                    shape: (256, 256, 1)         min:    0.00000  max:    1.00000  float32
molded_images            shape: (1, 1024, 1024, 1)    min:    0.00000  max:    0.99891  float64
image_metas              shape: (1, 15)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [46]:
im = '/notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm057.tif'

In [47]:
r2 = harz.detect_and_color_splash(model, im)

Running on /notebooks/tmp/data/bomb1_meiler2/bomb1_meiler2x_original/marsly_HG_Dgm057.tif
Processing 1 images
image                    shape: (256, 256, 1)         min:    0.00000  max:    1.00000  float32
molded_images            shape: (1, 1024, 1024, 1)    min:    0.00000  max:    0.99717  float64
image_metas              shape: (1, 15)               min:    0.00000  max: 1024.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32


In [48]:
r2.keys()

dict_keys(['rois', 'masks', 'class_ids', 'scores'])

In [27]:
r['rois'][0]

array([114, 115, 140, 141], dtype=int32)

In [28]:
out = np.zeros((len(r['scores']),256,256))
print(out.shape)
for i in range(len(r['scores'])):
    out[i][r['rois'][i][0]:r['rois'][i][2],r['rois'][i][1]:r['rois'][i][3]] = 1


(4, 256, 256)


In [49]:
out2 = np.zeros((len(r2['scores']),256,256))
print(out2.shape)
for i in range(len(r2['scores'])):
    out2[i][r2['rois'][i][0]:r2['rois'][i][2],r2['rois'][i][1]:r2['rois'][i][3]] = 1


(5, 256, 256)


In [29]:
np.unique(out[0],return_counts=True)

(array([0., 1.]), array([64860,   676]))

In [30]:
import src.my_utils as ut

In [22]:
r['rois']

array([[114, 115, 140, 141],
       [ 96,  46, 117,  67],
       [ 66,  78,  85,  97],
       [109,  90, 125, 105]], dtype=int32)

In [50]:
r2['rois']

array([[118, 119, 137, 138],
       [120,  38, 135,  54],
       [135,  46, 153,  65],
       [ 87,  28, 104,  45],
       [206, 197, 221, 213]], dtype=int32)

In [52]:
for i in range(5):
    ut.write_numpy_array_to_tif(out2[i], 'cc'+str(i)+'.tif', original_tif=im)

In [44]:
r['scores']

array([0.99524397, 0.97441554, 0.9380484 , 0.9263757 ], dtype=float32)

In [43]:
r2['scores']

array([0.99524397, 0.97441554, 0.9380484 , 0.9263757 ], dtype=float32)

In [24]:
r['class_ids']

array([2, 2, 2, 2], dtype=int32)

In [23]:
r['masks']

array([[[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        ...,
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        ...,
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        ...,
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       ...,

       [[False, False, False, False],
        [False, False, False, False],
        [False, False, False, False],
        ...,
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False, False]],

       [[False,